In [38]:
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

from sklearn.metrics import *
from sklearn.model_selection import *

import keras
from keras.layers import *
from keras.callbacks import *
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_x = pd.read_csv('X_train.csv')
train_y = pd.read_csv('y_train.csv')

num_test=500

def prepare_data(t):
    def f(d):
        d=d.sort_values(by=['measurement_number'])
        return pd.DataFrame({
         'lx':[ d['linear_acceleration_X'].values ],
         'ly':[ d['linear_acceleration_Y'].values ],
         'lz':[ d['linear_acceleration_Z'].values ],
         'ax':[ d['angular_velocity_X'].values ],
         'ay':[ d['angular_velocity_Y'].values ],
         'az':[ d['angular_velocity_Z'].values ],
         'ox':[ d['orientation_X'].values ],
         'oy':[ d['orientation_Y'].values ],
         'oz':[ d['orientation_Z'].values ],
         'ow':[ d['orientation_W'].values ],
        })

    t= t.groupby('series_id').apply(f)
    return t


def split_shuffle_groups(t):
    t= t.copy()

    # select randomly some groups (should be weighted by # of samples)

    aggcol='surface' # arbitrary; just to get size
    gstat= t.groupby('group_id')[aggcol].agg(np.size)
    gstat= gstat.reset_index()

    import random
    random.shuffle

    groups = list(zip(gstat['group_id'].values, gstat[aggcol].values))
    random.shuffle(groups)
    
    test_groups= set()
    c=0
    for gid,len in groups:
        if c>=num_test: break
        c+=len
        test_groups.add(gid)
    print("test groups:", test_groups)

    ctest = [ i for i,gid in enumerate(t['group_id']) if (gid in test_groups) ]
    ctrain = [ i for i,gid in enumerate(t['group_id']) if not (gid in test_groups) ]

    random.shuffle(ctrain)
    random.shuffle(ctest)

    return t.iloc[ctrain], t.iloc[ctest]


train= prepare_data(train_x)

# merge
train=pd.merge(train,train_y[['series_id','group_id','surface']],on='series_id')

train_part_df, validation_part_df= split_shuffle_groups(train)

print("training part of training data set:", train_part_df.describe())
print("validation part of training data set:",validation_part_df.describe())

test groups: {32, 35, 71, 72, 17, 55, 23, 56, 57, 31}
training part of training data set:          series_id     group_id
count  3294.000000  3294.000000
mean   1913.299939    35.960838
std    1103.993046    20.905975
min       0.000000     0.000000
25%     956.250000    16.000000
50%    1905.500000    39.000000
75%    2877.750000    53.000000
max    3808.000000    70.000000
validation part of training data set:          series_id    group_id
count   516.000000  516.000000
mean   1848.323643   48.073643
std    1073.482167   18.291979
min       1.000000   17.000000
25%     928.250000   32.000000
50%    1897.500000   55.000000
75%    2707.500000   71.000000
max    3809.000000   72.000000


In [26]:
x_test = pd.read_csv('X_train.csv')

In [2]:
validation_part_df

,series_id,lx,ly,lz,ax,ay,az,ox,oy,oz,ow,group_id,surface
2628,2628,"[0.14745, 1.2114, -1.8101, 2.0361, -1.0712, 1....","[2.5498, 5.5579, 3.9612, 5.3621, 4.9224, 3.258...","[-11.548, -7.0855, -5.9565, -6.8409, -8.8337, ...","[0.043367, 0.1186, 0.04458, -0.10388, -0.20994...","[-0.069471, -0.08884199999999999, -0.18097, -0...","[0.26804, 0.30964, 0.31049, 0.31574, 0.3181900...","[-0.82524, -0.82504, -0.82481, -0.82452, -0.82...","[0.54377, 0.5441199999999999, 0.54447, 0.5448,...","[0.08056100000000001, 0.080522, 0.080627999999...","[-0.12963, -0.12947999999999998, -0.12934, -0....",72,fine_concrete
469,469,"[1.6217, 0.24756999999999998, -0.24941, 2.1491...","[1.1562, 2.0635, 4.4366, -2.151, -1.5303, 1.26...","[-7.3521, -9.4797, -9.2318, -13.182, -6.1727, ...","[-0.12958, -0.073418, 0.023854, 0.15837, 0.315...","[-0.026679, -0.035188, -0.087376, -0.013944999...","[0.11597, 0.19744, 0.14802, 0.20371, 0.11715, ...","[-0.9316200000000001, -0.9315899999999999, -0....","[0.33104, 0.33119, 0.33138, 0.3316199999999999...","[0.043688, 0.04356, 0.043626, 0.04353999999999...","[-0.1435, -0.14336, -0.14331, -0.14313, -0.142...",31,concrete
3300,3300,"[1.9954, -0.65196, 2.3642, -0.22739, 1.5886, 2...","[4.1878, -2.3595, 0.0054093000000000006, 2.264...","[-9.6527, -20.158, -17.717, -9.02, -2.2387, -4...","[-0.41878999999999994, -0.090516, 0.33697, 0.4...","[0.08578200000000001, 0.079332, -0.04662300000...","[0.082067, 0.09305, 0.09328600000000001, 0.151...","[-0.19201, -0.19191, -0.1918, -0.19162, -0.191...","[0.96977, 0.96977, 0.96986, 0.96998, 0.9699, 0...","[0.14594000000000001, 0.14602, 0.14562, 0.1450...","[-0.0371, -0.037235000000000004, -0.0371149999...",55,tiled
3422,3422,"[-2.1498, 1.9005, 0.59815, -1.028, 1.9172, -1....","[2.2325, 2.8911, 2.3991, 1.7701, 1.9535, 3.174...","[-9.7426, -14.208, -14.82, -12.44, -9.4722, -7...","[-0.27052, -0.14615, 0.10253, 0.25609, 0.20797...","[0.071061, 0.11367000000000001, 0.064582, -0.1...","[-0.023413, -0.081189, -0.023516, -0.062397, -...","[-0.98369, -0.98377, -0.9837899999999999, -0.9...","[0.09563300000000001, 0.09545, 0.0954100000000...","[0.010346, 0.01022, 0.010133, 0.010253, 0.0102...","[-0.15197, -0.15161, -0.15149, -0.15116, -0.15...",71,tiled
975,975,"[0.51671, 2.6199, 0.35707, 1.5499, 1.8928, 0.9...","[1.0896, 1.2554, -0.07366, -1.4387, -0.79028, ...","[-10.556, -11.203, -10.77, -10.59, -9.3877, -8...","[0.019994, 0.056148, 0.08029, 0.1053, 0.050082...","[-0.0076503, 0.047849, 0.058948, -0.0177699999...","[-0.063974, -0.08369800000000001, -0.098178, -...","[0.18257, 0.18245999999999998, 0.18245, 0.1823...","[0.97139, 0.9714200000000001, 0.97145, 0.9715,...","[0.15024, 0.15018, 0.15002000000000001, 0.1498...","[0.022152, 0.022097, 0.021999, 0.022017, 0.022...",23,wood
897,897,"[4.1211, 1.7952, 1.4011, 3.33, 1.9953, 3.1982,...","[3.6248, 2.4712, 1.6578, 2.3394, 1.1946, 2.494...","[-7.4309, -7.5827, -8.511000000000001, -9.9257...","[0.10250999999999999, 0.025675, 0.005125999999...","[0.02995, 0.099353, 0.014488, -0.0183879999999...","[-0.041066000000000005, 0.017679, 0.020711, 0....","[-0.93448, -0.93452, -0.9345100000000001, -0.9...","[0.32465, 0.32449, 0.32452, 0.32461, 0.3246900...","[0.041491, 0.041266000000000004, 0.041243, 0.0...","[-0.14013, -0.14025, -0.14025, -0.14021, -0.14...",31,concrete
817,817,"[-2.3934, -0.029561, -0.95263, -0.351440000000...","[3.1577, 4.3292, 3.7604, 4.1804, 3.5919, 3.405...","[-9.1078, -7.0423, -8.0597, -9.3924, -10.109, ...","[0.014963999999999998, 0.012581, -0.045888, -0...","[0.096837, -0.0026241, -0.0076964, 0.063337, 0...","[-0.07029500000000001, -0.04835, -0.030614, -0...","[-0.87689, -0.8768600000000001, -0.87687, -0.8...","[0.45647, 0.45648, 0.45643999999999996, 0.4563...","[0.062571, 0.062697, 0.062737, 0.062731, 0.062...","[-0.13706, -0.13714, -0.13719, -0.13732, -0.13...",32,concrete
1988,1988,"[-0.88355, -2.4564, 0.37731, -2.7075, 1.9279, ...","[0.22794, -0.5461600000000001, 1.5533, 3.4089,...","[-4.8051, -10.242, -12.999, -11.72299999999999

In [3]:
train_part_df

,series_id,lx,ly,lz,ax,ay,az,ox,oy,oz,ow,group_id,surface
260,260,"[0.029968, -0.37043000000000004, 0.11159000000...","[3.7767, 3.3313, 2.9218, 2.9129, 3.6124, 3.355...","[-9.7082, -9.5806, -9.2043, -8.552999999999999...","[0.013304, 0.019181999999999998, 0.034688, 0.0...","[-0.0032619, 0.014925999999999998, 0.005596899...","[0.051807000000000006, 0.045659, 0.049296, 0.0...","[-0.39397, -0.39400999999999997, -0.39407, -0....","[-0.9071299999999999, -0.90712, -0.9071, -0.90...","[-0.13931, -0.13929, -0.13926, -0.13922, -0.13...","[-0.049968, -0.049944, -0.049912, -0.049898000...",34,soft_pvc
3137,3137,"[1.1205, 0.9694299999999999, 0.58537, 1.355, 0...","[3.7707, 3.7901, 4.9547, 5.4815, 5.2799, 5.144...","[-10.519, -10.512, -8.5463, -7.2911, -7.5823, ...","[-0.06433, 0.01798, 0.056298, 0.00584869999999...","[-0.00050664, -2.6822e-05, -0.027736, -0.01822...","[0.031363999999999996, 0.038666000000000006, 0...","[-0.48018999999999995, -0.48015, -0.4801, -0.4...","[0.86329, 0.8633200000000001, 0.86335, 0.86337...","[0.13193, 0.13191, 0.13189, 0.13189, 0.13194, ...","[-0.082137, -0.082133, -0.082069, -0.082051, -...",70,soft_pvc
1544,1544,"[-0.91789, 0.9882799999999999, 0.3264, -0.655,...","[3.884, 4.8103, 3.5693, 1.995, 2.5202, 3.313, ...","[-7.4599, -6.8648, -9.1525, -11.782, -12.23200...","[0.070626, -0.043119, -0.12036, -0.020619, 0.1...","[0.01329, 0.054254, 0.028576, -0.053535, -0.00...","[-0.032309, -0.044066, -0.011000000000000001, ...","[0.48693000000000003, 0.48682, 0.4867899999999...","[0.86005, 0.86008, 0.8600700000000001, 0.86007...","[0.13632, 0.13652999999999998, 0.13668, 0.1366...","[0.067982, 0.068031, 0.068075, 0.068149, 0.068...",19,soft_pvc
1898,1898,"[1.5978, -2.3515, -0.17465, -3.3068, -7.7386, ...","[2.0774, 0.78706, 0.047036, -0.86665, 1.656999...","[-6.3102, -7.5683, -10.142000000000001, -14.09...","[0.051999000000000004, 0.0015869999999999999, ...","[0.0040174, -0.036032999999999996, -0.086184, ...","[0.035496, 0.022767, 0.016946, 0.051278, 0.063...","[0.70111, 0.70113, 0.70117, 0.7013, 0.70138, 0...","[0.69707, 0.69705, 0.69703, 0.69697, 0.69692, ...","[0.11023, 0.1102, 0.11005999999999999, 0.10980...","[0.10191, 0.10192999999999999, 0.10194, 0.1017...",40,hard_tiles_large_space
1719,1719,"[0.0048524, 1.5427, 0.21679, 0.9356, 1.0446, 0...","[3.8216, 3.6176, 3.4087, 3.8061, 3.4992, 4.277...","[-8.4801, -8.7359, -8.9481, -9.1898, -9.5116, ...","[-0.030558999999999996, -0.038919999999999996,...","[-0.22430999999999998, -0.22024000000000002, -...","[0.75678, 0.75334, 0.77179, 0.74853, 0.76537, ...","[-0.9593, -0.9595, -0.9597399999999999, -0.959...","[-0.23820999999999998, -0.23735, -0.23638, -0....","[-0.045964, -0.045852, -0.045719, -0.045569, -...","[-0.14453, -0.14462, -0.14470999999999998, -0....",49,wood
3250,3250,"[-0.50949, 1.0034, -0.37044, 0.828770000000000...","[2.9322, 2.1886, 1.4411, 2.1407, 2.7148, 3.255...","[-8.6265, -10.097999999999999, -10.89, -9.8463...","[-0.039994999999999996, -0.032024000000000004,...","[0.023186000000000002, -0.00025928, 0.036864, ...","[-0.067143, -0.050068, -0.053234000000000004, ...","[-0.4915, -0.49155, -0.49163, -0.4917, -0.4917...","[0.8571799999999999, 0.85715, 0.85711, 0.85707...","[0.12966, 0.1297, 0.12966, 0.1296, 0.12959, 0....","[-0.08279199999999999, -0.082804, -0.082838, -...",70,soft_pvc
46,46,"[1.8162, 0.92156, 0.28704, 0.21936, -0.4101800...","[3.3952, 3.5282, 2.492, 2.9907, 3.2127, 2.8638...","[-9.9755, -9.7011, -9.4768, -8.6544, -8.4431, ...","[0.039355, 0.050460000000000005, 0.081085, 0.0...","[-0.074378, -0.053227, -0.028807999999999997, ...","[0.024386, 0.022665, 0.0088304, 0.022741, 0.03...","[0.84709, 0.8470700000000001, 0.84707999999999...","[-0.51079, -0.51085, -0.51088, -0.510910000000...","[-0.072277, -0.072275, -0.072253, -0.072239, -...","[0.12768, 0.12757000000000002, 0.12747, 0.1273...",26,soft_pvc
2803,2803,"[-0.29563, -0.6562600000000001, -1.3802, -0.95...","[3.6738, 3.8199, 3.9121, 4.4645, 4.56800000000...","[-8.9611, -10.157, -10.607000

In [4]:
encode_dic = {'fine_concrete': 0, 
              'concrete': 1, 
              'soft_tiles': 2, 
              'tiled': 3, 
              'soft_pvc': 4,
              'hard_tiles_large_space': 5, 
              'carpet': 6, 
              'hard_tiles': 7, 
              'wood': 8}

In [5]:
decode_dic = {0: 'fine_concrete',
              1: 'concrete',
              2: 'soft_tiles',
              3: 'tiled',
              4: 'soft_pvc',
              5: 'hard_tiles_large_space',
              6: 'carpet',
              7: 'hard_tiles',
              8: 'wood'}

In [6]:
y_train = train_part_df['surface']

In [7]:
y_train

260                   soft_pvc
3137                  soft_pvc
1544                  soft_pvc
1898    hard_tiles_large_space
1719                      wood
3250                  soft_pvc
46                    soft_pvc
2803                  soft_pvc
3339                  soft_pvc
797     hard_tiles_large_space
1860                  concrete
2341                  soft_pvc
826                 hard_tiles
30                  soft_tiles
719                   soft_pvc
3326                    carpet
78                    soft_pvc
3703                  concrete
1292                  concrete
807     hard_tiles_large_space
569                       wood
1741                     tiled
771                   concrete
1364                    carpet
1224                soft_tiles
1651                soft_tiles
2407                  concrete
2392                  soft_pvc
2009                    carpet
197                       wood
                 ...          
2249                      wood
40      

In [9]:
y_train = y_train.map(encode_dic).astype(int)

In [10]:
y_train

260     4
3137    4
1544    4
1898    5
1719    8
3250    4
46      4
2803    4
3339    4
797     5
1860    1
2341    4
826     7
30      2
719     4
3326    6
78      4
3703    1
1292    1
807     5
569     8
1741    3
771     1
1364    6
1224    2
1651    2
2407    1
2392    4
2009    6
197     8
       ..
2249    8
40      0
3613    1
3366    8
2129    0
2322    4
1971    5
2171    1
671     8
1855    3
158     4
2456    3
2793    3
2115    5
3761    1
251     8
643     1
3666    4
2416    0
3524    1
3316    8
2214    0
1850    4
821     8
3350    1
2004    5
1593    4
2382    5
308     6
314     2
Name: surface, Length: 3294, dtype: int32

In [11]:
validation_y = validation_part_df['surface']

In [12]:
validation_x = validation_part_df

In [14]:
validation_x

,series_id,lx,ly,lz,ax,ay,az,ox,oy,oz,ow,group_id,surface
2628,2628,"[0.14745, 1.2114, -1.8101, 2.0361, -1.0712, 1....","[2.5498, 5.5579, 3.9612, 5.3621, 4.9224, 3.258...","[-11.548, -7.0855, -5.9565, -6.8409, -8.8337, ...","[0.043367, 0.1186, 0.04458, -0.10388, -0.20994...","[-0.069471, -0.08884199999999999, -0.18097, -0...","[0.26804, 0.30964, 0.31049, 0.31574, 0.3181900...","[-0.82524, -0.82504, -0.82481, -0.82452, -0.82...","[0.54377, 0.5441199999999999, 0.54447, 0.5448,...","[0.08056100000000001, 0.080522, 0.080627999999...","[-0.12963, -0.12947999999999998, -0.12934, -0....",72,fine_concrete
469,469,"[1.6217, 0.24756999999999998, -0.24941, 2.1491...","[1.1562, 2.0635, 4.4366, -2.151, -1.5303, 1.26...","[-7.3521, -9.4797, -9.2318, -13.182, -6.1727, ...","[-0.12958, -0.073418, 0.023854, 0.15837, 0.315...","[-0.026679, -0.035188, -0.087376, -0.013944999...","[0.11597, 0.19744, 0.14802, 0.20371, 0.11715, ...","[-0.9316200000000001, -0.9315899999999999, -0....","[0.33104, 0.33119, 0.33138, 0.3316199999999999...","[0.043688, 0.04356, 0.043626, 0.04353999999999...","[-0.1435, -0.14336, -0.14331, -0.14313, -0.142...",31,concrete
3300,3300,"[1.9954, -0.65196, 2.3642, -0.22739, 1.5886, 2...","[4.1878, -2.3595, 0.0054093000000000006, 2.264...","[-9.6527, -20.158, -17.717, -9.02, -2.2387, -4...","[-0.41878999999999994, -0.090516, 0.33697, 0.4...","[0.08578200000000001, 0.079332, -0.04662300000...","[0.082067, 0.09305, 0.09328600000000001, 0.151...","[-0.19201, -0.19191, -0.1918, -0.19162, -0.191...","[0.96977, 0.96977, 0.96986, 0.96998, 0.9699, 0...","[0.14594000000000001, 0.14602, 0.14562, 0.1450...","[-0.0371, -0.037235000000000004, -0.0371149999...",55,tiled
3422,3422,"[-2.1498, 1.9005, 0.59815, -1.028, 1.9172, -1....","[2.2325, 2.8911, 2.3991, 1.7701, 1.9535, 3.174...","[-9.7426, -14.208, -14.82, -12.44, -9.4722, -7...","[-0.27052, -0.14615, 0.10253, 0.25609, 0.20797...","[0.071061, 0.11367000000000001, 0.064582, -0.1...","[-0.023413, -0.081189, -0.023516, -0.062397, -...","[-0.98369, -0.98377, -0.9837899999999999, -0.9...","[0.09563300000000001, 0.09545, 0.0954100000000...","[0.010346, 0.01022, 0.010133, 0.010253, 0.0102...","[-0.15197, -0.15161, -0.15149, -0.15116, -0.15...",71,tiled
975,975,"[0.51671, 2.6199, 0.35707, 1.5499, 1.8928, 0.9...","[1.0896, 1.2554, -0.07366, -1.4387, -0.79028, ...","[-10.556, -11.203, -10.77, -10.59, -9.3877, -8...","[0.019994, 0.056148, 0.08029, 0.1053, 0.050082...","[-0.0076503, 0.047849, 0.058948, -0.0177699999...","[-0.063974, -0.08369800000000001, -0.098178, -...","[0.18257, 0.18245999999999998, 0.18245, 0.1823...","[0.97139, 0.9714200000000001, 0.97145, 0.9715,...","[0.15024, 0.15018, 0.15002000000000001, 0.1498...","[0.022152, 0.022097, 0.021999, 0.022017, 0.022...",23,wood
897,897,"[4.1211, 1.7952, 1.4011, 3.33, 1.9953, 3.1982,...","[3.6248, 2.4712, 1.6578, 2.3394, 1.1946, 2.494...","[-7.4309, -7.5827, -8.511000000000001, -9.9257...","[0.10250999999999999, 0.025675, 0.005125999999...","[0.02995, 0.099353, 0.014488, -0.0183879999999...","[-0.041066000000000005, 0.017679, 0.020711, 0....","[-0.93448, -0.93452, -0.9345100000000001, -0.9...","[0.32465, 0.32449, 0.32452, 0.32461, 0.3246900...","[0.041491, 0.041266000000000004, 0.041243, 0.0...","[-0.14013, -0.14025, -0.14025, -0.14021, -0.14...",31,concrete
817,817,"[-2.3934, -0.029561, -0.95263, -0.351440000000...","[3.1577, 4.3292, 3.7604, 4.1804, 3.5919, 3.405...","[-9.1078, -7.0423, -8.0597, -9.3924, -10.109, ...","[0.014963999999999998, 0.012581, -0.045888, -0...","[0.096837, -0.0026241, -0.0076964, 0.063337, 0...","[-0.07029500000000001, -0.04835, -0.030614, -0...","[-0.87689, -0.8768600000000001, -0.87687, -0.8...","[0.45647, 0.45648, 0.45643999999999996, 0.4563...","[0.062571, 0.062697, 0.062737, 0.062731, 0.062...","[-0.13706, -0.13714, -0.13719, -0.13732, -0.13...",32,concrete
1988,1988,"[-0.88355, -2.4564, 0.37731, -2.7075, 1.9279, ...","[0.22794, -0.5461600000000001, 1.5533, 3.4089,...","[-4.8051, -10.242, -12.999, -11.72299999999999

In [15]:
validation_y = validation_y.map(encode_dic).astype(int)

In [16]:
validation_y

2628    0
469     1
3300    3
3422    3
975     8
897     1
817     1
1988    3
145     2
507     0
1668    1
2251    1
2203    3
1118    0
1689    8
2653    4
1839    3
1736    1
442     8
2313    0
1383    8
2231    3
3342    0
1440    8
1931    0
2159    3
3292    1
2179    3
1663    1
1638    1
       ..
2771    0
2080    1
1006    1
716     0
885     1
524     1
2575    0
705     0
3640    3
2438    1
3157    3
601     0
2853    3
1705    3
3441    3
2579    1
585     1
2734    3
1499    8
3370    0
483     8
829     8
3064    3
2386    4
1023    1
1919    1
2395    3
617     1
57      0
3809    4
Name: surface, Length: 516, dtype: int32

In [27]:
x_train = train_part_df

In [32]:
# del x_train['series_id']
# del x_train['group_id']
# del x_train['surface']
del x_test['row_id']
del x_test['series_id']
del x_test['measurement_number']

In [34]:
x_train

,lx,ly,lz,ax,ay,az,ox,oy,oz,ow
260,"[0.029968, -0.37043000000000004, 0.11159000000...","[3.7767, 3.3313, 2.9218, 2.9129, 3.6124, 3.355...","[-9.7082, -9.5806, -9.2043, -8.552999999999999...","[0.013304, 0.019181999999999998, 0.034688, 0.0...","[-0.0032619, 0.014925999999999998, 0.005596899...","[0.051807000000000006, 0.045659, 0.049296, 0.0...","[-0.39397, -0.39400999999999997, -0.39407, -0....","[-0.9071299999999999, -0.90712, -0.9071, -0.90...","[-0.13931, -0.13929, -0.13926, -0.13922, -0.13...","[-0.049968, -0.049944, -0.049912, -0.049898000..."
3137,"[1.1205, 0.9694299999999999, 0.58537, 1.355, 0...","[3.7707, 3.7901, 4.9547, 5.4815, 5.2799, 5.144...","[-10.519, -10.512, -8.5463, -7.2911, -7.5823, ...","[-0.06433, 0.01798, 0.056298, 0.00584869999999...","[-0.00050664, -2.6822e-05, -0.027736, -0.01822...","[0.031363999999999996, 0.038666000000000006, 0...","[-0.48018999999999995, -0.48015, -0.4801, -0.4...","[0.86329, 0.8633200000000001, 0.86335, 0.86337...","[0.13193, 0.13191, 0.13189, 0.13189, 0.13194, ...","[-0.082137, -0.082133, -0.082069, -0.082051, -..."
1544,"[-0.91789, 0.9882799999999999, 0.3264, -0.655,...","[3.884, 4.8103, 3.5693, 1.995, 2.5202, 3.313, ...","[-7.4599, -6.8648, -9.1525, -11.782, -12.23200...","[0.070626, -0.043119, -0.12036, -0.020619, 0.1...","[0.01329, 0.054254, 0.028576, -0.053535, -0.00...","[-0.032309, -0.044066, -0.011000000000000001, ...","[0.48693000000000003, 0.48682, 0.4867899999999...","[0.86005, 0.86008, 0.8600700000000001, 0.86007...","[0.13632, 0.13652999999999998, 0.13668, 0.1366...","[0.067982, 0.068031, 0.068075, 0.068149, 0.068..."
1898,"[1.5978, -2.3515, -0.17465, -3.3068, -7.7386, ...","[2.0774, 0.78706, 0.047036, -0.86665, 1.656999...","[-6.3102, -7.5683, -10.142000000000001, -14.09...","[0.051999000000000004, 0.0015869999999999999, ...","[0.0040174, -0.036032999999999996, -0.086184, ...","[0.035496, 0.022767, 0.016946, 0.051278, 0.063...","[0.70111, 0.70113, 0.70117, 0.7013, 0.70138, 0...","[0.69707, 0.69705, 0.69703, 0.69697, 0.69692, ...","[0.11023, 0.1102, 0.11005999999999999, 0.10980...","[0.10191, 0.10192999999999999, 0.10194, 0.1017..."
1719,"[0.0048524, 1.5427, 0.21679, 0.9356, 1.0446, 0...","[3.8216, 3.6176, 3.4087, 3.8061, 3.4992, 4.277...","[-8.4801, -8.7359, -8.9481, -9.1898, -9.5116, ...","[-0.030558999999999996, -0.038919999999999996,...","[-0.22430999999999998, -0.22024000000000002, -...","[0.75678, 0.75334, 0.77179, 0.74853, 0.76537, ...","[-0.9593, -0.9595, -0.9597399999999999, -0.959...","[-0.23820999999999998, -0.23735, -0.23638, -0....","[-0.045964, -0.045852, -0.045719, -0.045569, -...","[-0.14453, -0.14462, -0.14470999999999998, -0...."
3250,"[-0.50949, 1.0034, -0.37044, 0.828770000000000...","[2.9322, 2.1886, 1.4411, 2.1407, 2.7148, 3.255...","[-8.6265, -10.097999999999999, -10.89, -9.8463...","[-0.039994999999999996, -0.032024000000000004,...","[0.023186000000000002, -0.00025928, 0.036864, ...","[-0.067143, -0.050068, -0.053234000000000004, ...","[-0.4915, -0.49155, -0.49163, -0.4917, -0.4917...","[0.8571799999999999, 0.85715, 0.85711, 0.85707...","[0.12966, 0.1297, 0.12966, 0.1296, 0.12959, 0....","[-0.08279199999999999, -0.082804, -0.082838, -..."
46,"[1.8162, 0.92156, 0.28704, 0.21936, -0.4101800...","[3.3952, 3.5282, 2.492, 2.9907, 3.2127, 2.8638...","[-9.9755, -9.7011, -9.4768, -8.6544, -8.4431, ...","[0.039355, 0.050460000000000005, 0.081085, 0.0...","[-0.074378, -0.053227, -0.028807999999999997, ...","[0.024386, 0.022665, 0.0088304, 0.022741, 0.03...","[0.84709, 0.8470700000000001, 0.84707999999999...","[-0.51079, -0.51085, -0.51088, -0.510910000000...","[-0.072277, -0.072275, -0.072253, -0.072239, -...","[0.12768, 0.12757000000000002, 0.12747, 0.1273..."
2803,"[-0.29563, -0.6562600000000001, -1.3802, -0.95...","[3.6738, 3.8199, 3.9121, 4.4645, 4.56800000000...","[-8.9611, -10.157, -10.607000000000001, -9.963...","[-0.066441, -0.045156, -0.01702, 0.00666530000...","[-0.0026449999999999998, 0.00026822, -0.006729...","[-0.009939100000000001, 0.00728

In [35]:
def make_model():
    inp = Input(shape=(128, 10))
    x = Bidirectional(CuDNNLSTM(32, return_sequences=True))(inp)
#     x = Attention(128)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(.5)(x)
    x = Dense(9, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [36]:
def k_folds(X, y, X_test, k=5):
    folds = list(StratifiedKFold(n_splits=k).split(X, y))
    y_test = np.zeros((X_test.shape[0], 9))
    y_oof = np.zeros((X.shape[0]))
    
    for i, (train_idx, val_idx) in  enumerate(folds):
        print(f'Fold {i+1}')
        model = make_model()
        model.fit(X[train_idx], y[train_idx], batch_size=128, epochs=100, 
                  validation_data=[validation_x, validation_y], verbose=0)
        
        pred_val = np.argmax(model.predict(X[val_idx]), axis=1)
        score = accuracy_score(pred_val, y[val_idx])
        y_oof[val_idx] = pred_val
        
        print(f'Scored {score:.3f} on validation data')
        
        y_test += model.predict(X_test)
        
    return y_oof, y_test     

In [39]:
y_oof, y_test = k_folds(x_train, y_train, x_test, k=5)

Fold 1


KeyError: "None of [Int64Index([ 573,  577,  583,  585,  586,  590,  591,  592,  593,  594,\n            ...\n            3284, 3285, 3286, 3287, 3288, 3289, 3290, 3291, 3292, 3293],\n           dtype='int64', length=2631)] are in the [columns]"